# **CPES 2 : TP7 - Traitement de données spectroscopiques**

*Année 2022-2023*
***

**Noms :**
<br>
<br>
**Prénoms :**


Le but de la séance est d'aborder les différentes notions et étapes du traitement de données dans le cadre d'observations spectroscopiques, ainsi que l'analyse scientifique d'un spectre d'étoile et son interprétation.

<font color='red'> **Ce notebook fait office de compte-rendu. Vous compléterez le code et rédigerez vos réponses directement dessus, afin de rendre un notebook par groupe à la fin de la séance.**

**IMPORTANT :** Ne pas oublier de mettre vos noms dans le nom du fichier. Exemple : *TP7_CPES2_nom1_nom2.ipynb*
    

## **1 - Objectif**


##### <font color='blue'> **Première utilisation du notebook : guide d'installation**

Pour télécharger ce notebook, aller à l'adresse suivante : https://share.obspm.fr/s/EERtZGFxrkEpkxR. Le lien contient une archive contenant le notebook et les données dont vous avez besoin.
<br>
Commencer par extraire l'archive dans *Documents* puis ouvrer un terminal avec Ctrl+Alt+T puis taper la commande suivante :
<br>
```
$ cd Documents/TP7_CPES2/
```

Pour que le notebook fonctionne correctement, celui-ci doit être lancé via **Jupyter Lab** via le terminal avec la commande :

```
$ jupyter lab
```

Si un module est manquant, il peut également être installé via pip depuis le terminal avec :


```
$ pip install numpy
```

***

## **2 - Etalonnage précis des données spectroscopiques**


La première étape avant d'analyser un spectre est de l'étalonner précisément, on parle également de "réduction" ou de "traitement" de données.

### **2.1 Rappel rapide du traitement de données**

Quelques rappels sur le traitement de données que vous avez vu l'année dernière. La relation qui donne l'intensité finale enregistrée sur l'ordinateur en fonction du signal lumineux incident peut simplement être décrite par l'équation suivante :

\begin{equation}
    \label{eq_1} \tag{1}
    I_{mesure}(ADU)=T_{atmo} \cdot T_{intrument} \cdot Signal + biais + courant_{obscurite}(t)
\end{equation} 


Avec :
- $I_{mesure}(ADU, Analog-to-Digital~Unit)$ la mesure finale numérisée que vous allez étudier dans ce TP.
- $T_{atmo}$ le coefficient de transmission de l'atmosphère. Il ne s'applique que si le signal traverse l'atmosphère terrestre.
- $T_{intrument}$ le coefficient de transmission de l'instrument : optique du télescope et du spectro, réponse de la camera, etc.
- $Signal$ le signal lumineux en entrée de l'instrument (lumière provenant de l'étoile, luminosité du ciel, pollution lumineuse, etc).
- $biais$ le niveau moyen qui est généralement introduit au niveau du détecteur (ici, la caméra) pour éviter les valeurs négatives en ADU.
- $courant_{obscurite}(t)$ le courant d'obscurité de la caméra de détection. C'est un niveau moyen qui varie proportionnellement au temps de pose et qui augmente avec la température du détecteur.

On appelle FLAT la mesure effectuée dans le but d'appliquer la correction du champ plat correspondant à l'instrument, et on parle de DARK pour décrire la mesure simultanée du courant d'obscurité et du biais de la caméra.

##### **Question 1 (1 pt) :**

A partir de l'équation (1), quels sont les deux mesures que vous devez faire pour déterminer $T_{atmo}.T_{intrument}$ et $biais + courtant_{obscurite}(t)$ ? <br> Laquelle correspond au FLAT et laquelle correspond au DARK ?

> <font color='blue'> Répondre ici :


##### **Question 2 (1 pt) :**
Rappeler le principe et décrire la procédure d'acquisition du FLAT en imagerie.

> <font color='blue'> Répondre ici :


##### **Question 3 (1 pt) :**
Pourquoi peut-on supposer que $T_{atmo}=1$ lorsque l'on enregistre le FLAT?

> <font color='blue'> Répondre ici :


##### **Question 4 (1 pt):**
Quelle est la particularité des FLATS en spectroscopie par rapport à l'imagerie ? Quelle information faut-il connaître sur la source utilisée pour enregistrer l'image de FLAT ?

> <font color='blue'> Répondre ici : 

##### **Question 5 (1 pt) :**
On rappelle que le flat doit-être normalisé. Qu'est-ce que cela signifie, et pourquoi cela est-il important ?

> <font color='blue'> Répondre ici : 

##### **Question 6 (1 pt) :**
Rappeler le principe et décrire la procédure d'acquisition du DARK.

> <font color='blue'> Répondre ici : 

##### **Question 7 (1 pt) :**
Qu'est qu'un FOND DE CIEL (aussi appelé SKY) et qu'est ce qui le différencie d'un DARK ?

> <font color='blue'> Répondre ici : 

##### **Question 8 (1 pt) :**
Au cours de l'observation, quand doit-on enregistrer les FLATS, les DARKS et les FONDS DE CIEL ?

> <font color='blue'> Répondre ici : 

Au cours de la séance d'observation, on réalise plusieurs séries de FLATS et de DARKS/FONDS DE CIEL. Cela permet de faire des moyennes appelées "master flat" et "master dark/fond de ciel"). Cela permet également d'éliminer certains FLATS ou DARKS/FONDS DE CIEL qui ne sont pas satisfaisant.

##### **Question 9 (1 pt) :**
Décrire le traitement des données que vous allez faire pour déterminer  $T_{intrument}$.
<br>
**Faire valider la formule par un encadrant avant de passer à la suite.**

> <font color='blue'> Répondre ici : 

##### **Question 10 (1 pt) :**
Proposer des solutions pour déterminer $T_{atmo}$.

> <font color='blue'> Répondre ici : 

### **2.2 Application aux données du TP**

Nous allons maintenant appliquer ces principes pour réduire un spectre d'une étoile. Nous allons pour cela utiliser différentes librairies python permettant d'ouvrir des données astronomiques, de les manipuler et de les visualiser. Les données brutes ainsi que les fichiers d'étalonnage se trouvent dans le dossier "Exercice1".

#### **2.2.1 Chargement des données**

Dans cette sous-partie nous allons nous intéresser aux DARKS et au FONDS DE CIEL, ainsi qu'à la façon dont on obtient un master DARK.

***

La première étape consiste à charger les différents modules python dont nous aurons besoin tout au long de ce TP :
- *os* (https://docs.python.org/3/library/os.html) dispose d'outils pour naviguer dans le système et lister les fichiers d'un dossier de l'ordinateur
- *numpy* (https://numpy.org/doc/stable/) permet de manipuler des tableaux de données à plusieurs dimensions et d'effectuer toute sorte d'opérations (sommes, multiplications matricielles, moyennes...)
- *matplotib* (https://matplotlib.org/stable/index.html) offre un large panel d'outils permettant de visualiser les données et de produire des graphiques
- *astropy* (https://docs.astropy.org/en/stable/index.html) est une librairie dédiée à l'astronomie et l'astrophysique et contient une multitude d'outils pratiques pour la traitement des données. Nous utilisons surtout ici la fonction de lecture des fichiers au format FITS.
- *scipy* (https://docs.scipy.org/doc/scipy/) est une librairie (basée sur numpy) qui contient un large éventail de fonctions mathématique.

Note : ne pas hésiter à consulter la documentation de ces différents modules en cas de besoin. De nombreux exemples d'utilisation y sont présentés.

In [ ]:
# Importation des modules
import os
import numpy as np               # on peut renommer un module avec une variable plus courte avec le mot clé "as"
import matplotlib.pyplot as plt  

from astropy.io import fits      # "from" permet de ne charger que le sous-module "fits" de la librairie "astropy" afin de ne pas encombrer inutilement la mémoire
from scipy import interpolate    # On importe les fonctions d'interpolation du module scipy

# commande "%magic" pour utiliser le backend "widget" de matplotlib permettant d'avoir les outils interactifs (tels que le zoom) sur les figures
# Cette ligne peut être commentée en cas d'erreur lors de l'affichage des graphes de matplotlib.
%matplotlib widget               

Dans Jupyter Lab, une commande très utile est le **?** : placé derrière une commande, il permet d'accéder rapidement à sa documentation pour savoir comment l'utiliser. Exemple avec la fonction *print( )* :

In [ ]:
print?

<br>Il suffit donc de créer une nouvelle cellule, et d'écrire le nom de la fonction en remplaçant les parenthèses **( )** par le point d'interrogation **?** pour obtenir des informations sur l'utilisation de la fonction.

***

Nous allons maintenant charger les données au format "FITS" à l'aide du module astropy.io.fits (que nous avons renommé "fits" pour plus de simplicité).
Commençons par vérifier que nos fichiers de données se trouvent bien à l'endroit attendu : la commande "os.listdir(*path*)" permet d'obtenir la liste de fichiers présents dans le répertoire indiqué par la variable *path*, et la commande "print(*text*)" permet d'afficher le *text* indiqué.

In [ ]:
path = './Data_TP7_CPES2/Exercice1/'
file_list = os.listdir(path)
print(file_list)

<br>Nous avons maintenant stocké la liste de fichiers présents dans le répertoire "Exercice1" dans notre variable "file_list". Nous allons pouvoir parcourir cette liste avec une boucle "for" afin de charger ces fichiers dans le code.

In [ ]:
loaded_files = {}

for file_name in file_list:
    try:
        current_fits = fits.open(path+file_name)[0] # fits.open() renvoie une liste de tableaux de données (appelés "HDU") contenus dans le fichier FITS. Ici nos fichiers ne contiennent qu'un seul HDU, que l'on sélectionne avec l'indice [0]
        loaded_files[file_name] = current_fits      # on stocke ici le fichier .fits chargé dans un dictionnaire afin d'y accéder plus facilement par la suite en utilisant le nom du fichier d'origine
        print(f'{file_name:<80} has been loaded !')
    except Exception as e:                          # En cas d'echec de chargement du fichier, la structure try:... except:... permet d'empêcher au code de planter en affichant l'erreur et en passant au fichier suivant (par exemple si le fichier n'est pas au format .fits)
        print(f'Can\'t load {file_name:<69} {e}')

<br>
Tous les fichiers 'FITS' contenus dans le dossier 'Exercice1' ont été chargés et stockés dans le dictionnaire "loaded_files". Un dictionnaire permet de stocker différentes 'valeurs' et d'y accéder grâce à la 'clé' sous laquelle la valeur a été enregistrée. Par exemple, considerons le dictionnaire suivant :


```python
dico = {
    'key_1' : 42,
    'key_2' : aze,
    'key_3' : jpp
}
```

La valeur "42" a été stockée via la clé 'key_1'. On peut donc accéder à cette valeur via la commande suivante :

```python
print(dico['key_1'])
42
```

Qui affichera donc le contenu du dictionnaire "dico" correspondant à la clé 'key_1', c'est-à-dire 42. Dans notre cas, les fichiers ont tous été chargés dans un dictionnaire "loaded_files" avec pour chacun une clé correspondant au nom du fichier. On accédera donc par exemple aux données du fichier "dark_20s_1.fits" par la commande :

In [ ]:
file = loaded_files['dark_20s_1.fits']
print(file)

<br>
On voit ici que "loaded_files['dark_20s_1.fits']" permet bien d'accéder à un objet de type "astropy.io.fits", c'est-à-dire un fichier FITS chargé dans le code python et que nous pouvons maintenant manipuler avec tous les outils python habituels. On pourra par exemple accéder aux données du fichier fits grâce à l'attribut '.data', qui renvoie les données du fichier FITS dans un tableau de données numpy :

In [ ]:
file = loaded_files['dark_20s_1.fits']
print(file.data)

<br>On peut alors utiliser le module *matplotlib.pyplot* (renommé ici *plt*) afin d'afficher ces données sur un graphique :

In [ ]:
file = loaded_files['dark_20s_1.fits']
plt.figure()
plt.plot(file.data)

Le format FITS est un format très utilisé pour les images en astrophysique. Il est également pratique pour d'autres types de données tels que les spectres. Il permet de stocker des métadonnées concernant les images dans ce qu'on appelle le "header" (entête) du FITS. On accède ici au header d'un fichier FITS grâce à l'attribut '.header' :

In [ ]:
file = loaded_files['dark_20s_1.fits']
file.header

<br> Le header d'un fichier FITS est également donné sous la forme d'un dictionnaire. Dans ce cas, le format d'affichage est de la forme "Key = Value" : par exemple dans notre cas, le temps de pose (en ms) correspond à la clé "EXP (MS)". On accède donc au temps de pose utilisé pour le fichier "dark_20s_1.fits" avec la commande : 

In [ ]:
file = loaded_files['dark_20s_1.fits']
print(file.header['EXP (MS)'])

<br> Pour accéder aux données ou header des autres fichiers, il suffit donc de changer le nom du fichier entre [ ] après loaded_files.

#### **2.2.2 Construction du spectre de dark maître ou "MASTER DARK"**

Dans cette sous-partie nous allons nous intéresser aux DARKS et aux FONDS DE CIEL, ainsi qu'à la façon dont on obtient un MASTER DARK.

##### **Question 11 (1 pt) :**
Compléter le code suivant afin d'afficher les temps de pose appliqués à chacun des différents fichiers, et vérifier qu'ils correspondent bien à la valeur indiquée dans le nom du fichier.

In [ ]:
# CODE A COMPLETER
for name,file in loaded_files.items():
    temps_pose =  # ligne à completer
    print(f'Temps de pose de {name:<60} : {temps_pose} ms')

> <font color='blue'> Répondre ici : 

<br> <font color='red'> Attention : une fois le fichier renommé (et de même pour toute opération sur les fichiers), il faudra relancer le notebook en entier pour recharger toutes les données correctement en appuyant sur le bouton *Restart Kernel and Run all Cells...* (double flèche vers la droite).

##### **Question 12 (1 pt) :**
Compléter le code suivant afin d'afficher pour chaque fichier la température de la caméra en (°C) (**Tempx10** est la température en degrés Celsius multipliée par 10). La température est-elle la même pour tous les darks effectués? Pourquoi la caméra est-elle refroidie?

In [ ]:
# CODE A COMPLETER
for name,file in loaded_files.items():
    temperature =  # ligne à completer
    print(f'Température de la caméra pour {name:<60} : {temperature} °C')

> <font color='blue'> Répondre ici : 

##### **Question 13 (1 pt) :**
Compléter le code suivant afin d'afficher tous les fichiers dark sur un même graphique. Décrire les darks et les différences qu'il y a entre eux. Quelles sont les raisons de ces différences ?

In [ ]:
# CODE A COMPLETER
plt.figure()
for name,file in loaded_files.items():
    if 'dark' in  name: # on ne sélectionne que les fichiers contenant "dark" dans leur nom
        spectre =  # ligne à completer
        plt.plot(spectre,label=name)
    else:
        pass
    
plt.title('Comparaison des darks')
plt.ylabel('Flux (ADU)')
plt.xlabel('pixel')
plt.legend()

> <font color='blue'> Répondre ici : 

##### **Question 14 (3 pts) :**
Nous allons réaliser un dark moyen de 1 seconde. Pour cela, nous avons vu comment accéder aux données d'un fichier FITS à partir de notre dictionnaire *loaded_files* en utilisant le nom du fichier et l'attribut ".data":

```python
    data_1 = loaded_files["fichier_1.fits"].data
    data_2 = loaded_files["fichier_2.fits"].data
``` 

On récupère alors dans *data_1* et *data_2* les données des fichiers "fichier_1.fits" et "fichier_2.fits". *data_1* et *data_2* sont des tableaux de données au format *numpy*, ce qui permet d'effectuer très simplement tout un tas d'opérations entre ces deux jeux de données. Par exemple pour calculer leur moyenne, il suffit de faire :

```python
data_moy = (data_1 + data_2) / 2
```

De manière plus élégante et surtout plus rapide, on peut utiliser les fonctions numpy (le gain de temps quand il y a beaucoup de données peut être très important) :

```python
data_array = np.array([data_1, data_2]) # data_array est un tableau 2D, le premier indice/axis correspond à un sous tableau (data_array[0] contient data_1) 
data_moy   = np.mean(data_array, axis=0) # on laisse numpy calculer la moyenne, en précisant de ne la faire que sur les "lignes" du gros tableau data_array (avec axis=0) càd en sommant les dark entre eux
```

**En s'inspirant de cet exemple, écrire un code ci-dessous pour calculer un dark moyen de 1 seconde. Vous nommerez la variable du dark moyen "master_dark_1s".**

In [ ]:
# CODE A COMPLETER


On enregistre ensuite le master_dark_1s afin de pouvoir le réutiliser plus tard. Ceci est fait grâce à la fonction np.save(*name*,a), qui sauvegarde un tableau *numpy* "a" sous le nom "name" indiqué. Complétez le code ci-dessous en entrant vos noms et prénoms dans le nom du fichier et executez la cellule pour sauvegarder votre master_dark_1s :

In [ ]:
# CODE A COMPLETER
name =  # ligne à modifier
np.savetxt(name,master_dark_1s)
print(f'{name} a bien été sauvegardé !')

**De même, écrire un code ci-dessous permettant de calculer et d'enregistrer un dark moyen de 20 secondes et un fond de ciel moyen. Vous nommerez les variables respectivement "master_dark_20s" et "master_fond_ciel_20s", et les enregistrerez sous la forme "nom_prenom_exo1_MASTER_DARK_20s.dat" et "nom_prenom_exo1_MASTER_FOND_DE_CIEL_20s.dat"**

In [ ]:
[name for name in file_list if 'fond' in name]

In [ ]:
# CODE A COMPLETER : calculer le master_dark_20s


In [ ]:
# CODE A COMPLETER : enregistrer le master_dark_20s sous le nom "nom_prenom_exo1_MASTER_DARK_20s.dat"

name = 
np.savetxt(name,master_dark_20s)
print(f'{name} a bien été sauvegardé !')

In [ ]:
# CODE A COMPLETER : calculer le master_fond_ciel


In [ ]:
# CODE A COMPLETER : enregistrer le master_fond_ciel sous le nom "nom_prenom_exo1_MASTER_FOND_DE_CIEL.dat"

name = 
np.savetxt(name,master_dark_20s)
print(f'{name} a bien été sauvegardé !')

**La cellule ci-dessous permet de comparer les master dark de 1s et 20s. Décrire ces deux master darks : y-a-t-il des variations ?**

In [ ]:
plt.figure()                                      # Créer une figure vide
plt.plot(master_dark_1s,label='master_dark_1s')   # Trace la courbe master_dark_1s en fonction des pixels
plt.plot(master_dark_20s,label='master_dark_20s') # Trace la courbe master_dark_20s en fonction des pixels
plt.ylabel('Flux (ADU)')                          # Ajoute le label "Flux" sur l'axe 'y' 
plt.xlabel('pixel')                               # Ajoute le label "pixel" sur l'axe 'x' 
plt.title('Comparaison des master dark')          # Ajoute un titre à la figure
plt.legend()                                      # Affiche la legende avec le code coleur des courbes

> <font color='blue'> Répondre ici : 

##### **Question 15 (1 pt) :**
Comparer le master dark de 20 secondes avec un dark simple de 20 secondes, quels sont les différences ? Pourquoi ?
<br>
Vous pourrez faire la comparaison avec plusieurs dark simple de 20 secondes.

In [ ]:
# CODE A COMPLETER : afficher sur une même figure le master_dark_20s et un dark_20s_x


> <font color='blue'> Répondre ici : 

##### **Question 16 (2 pts) :**
Comparer maintenant le master dark de 20 secondes avec le master fond de ciel de 20 secondes, quels sont les différences ? Pourquoi ?

In [ ]:
# CODE A COMPLETER : afficher sur une même figure le master_dark_20s et le master_fond_de_ciel_20s


> <font color='blue'> Répondre ici : 

#### **2.2.3 Construction du spectre de champ plat maître ou "MASTER FLAT"**

Nous allons maintenant voir comment construire un spectre de champ plat (ou flat-field, que nous appellerons FLAT par la suite) afin d'étalonner la réponse du détecteur. Pour cet exercice nous utiliserons les données qui se trouvent dans le dossier "exercice2".
<br>
Pour rappel, les FLATS sont mesurés pendant les observations à l'aide de la lampe du même nom. Le rôle du FLAT est de corriger l'inhomogénéité des pixels du capteur ainsi que la transmission qui varie en fonction de la longueur d'onde. En théorie, on prend des images d'une source lumineuse uniforme en longueur d'onde et spatialement (sur tout le détecteur). En réalité, la source lumineuse est un corps noir à environ 4000K qui n'est donc pas uniforme en longueur d'onde. Le FLAT aura
donc besoin d'être corrigé de ce "problème" en le divisant par une courbe de corps noir à 4000K.

In [ ]:
path = './Data_TP7_CPES2/Exercice2/'
file_list = os.listdir(path)

In [ ]:
loaded_files = {}

for file_name in file_list:
    try:
        current_fits = fits.open(path+file_name)[0] # fits.open() renvoie une list de tableaux de données (appelés "HDU") contenus dans le fichier FITS. Ici nos fichiers ne contiennent qu'un seul HDU, que l'on sélectionne avec l'indice [0]
        loaded_files[file_name] = current_fits      # on stock ici le fichier .fits chargé dans un dictionnaire afin d'y accéder plus facilement par la suite en utilisant le nom du fichier d'origine
        print(f'{file_name:<80} has been loaded !')
    except Exception as e:                          # En cas d'echec de chargement du fichier, la structure try:... except:... permet d'empêcher au code de planter en affichant l'erreur et en passant au fichier suivant (par exemple si le fichier n'est pas au format .fits)
        print(f'Can\'t load {file_name:<69} {e}')

In [ ]:
# récupération des longueurs d'onde du spectre à partir du header
lmin = float(loaded_files['flat_5s_1.fits'].header['CRVAL1']) # longueur d'onde du premier point en nm
nb   = int(loaded_files['flat_5s_1.fits'].header['NAXIS1'])   # nombre de canaux d'échantillonnage
step = float(loaded_files['flat_5s_1.fits'].header['CD1_1'])  # écart entre deux échantillon en nm

wave = np.linspace(lmin-nb*step/2,lmin+nb*step/2,nb)

##### **Question 17 (1 pt) :**
Quelle formule va-t-on appliquer pour construire le master flat ?
<br>
**Faire valider la formule par un encadrant avant de passer à la suite.**

> <font color='blue'> Répondre ici : 

##### **Question 18 (1 pt) :**
De la même manière que vous avez construit les masters dark en faisant la moyenne de plusieurs darks, construire un flat moyen que vous nommerez *master_flat_5s*.




In [ ]:
# CODE A COMPLETER : calculer le master_flat_5s


In [ ]:
plt.figure()
plt.plot(wave,mean_flat_5s,label='mean_flat_5s')
plt.ylabel('Flux (ADU)')
plt.xlabel('Wavelength (nm)')
plt.title('Flat moyen')
plt.legend()

**Pourquoi le spectre de flat a t-il cette forme ?**

> <font color='blue'> Répondre ici : 

##### **Question 19 (2 pts) :**
Le flat doit être corrigé du dark, cependant les temps de pose pour prendre les flats peuvent être différents de ceux utilisés pour les données scientifiques, il faut donc un master dark spécifique aux flats. La source utilisée pour faire le flat étant un corps noir à 4000K, elle n'est pas spectralement homogène. Il faut donc aussi corriger l'effet du corps noir sur le flat.
<br>
**Construire le master dark associé au flat moyen puis le master flat corrigé.**

In [ ]:
# CODE A COMPLETER : calculer le master_dark_5s


In [ ]:
# La taille du tableau du corps noir n'est pas la même que celle des données
# On fait donc une interpolation puis on crée un tableau de même taille que nos données

CN4000K = loaded_files["corps_noir_4000K.fits"].data

# On récupère les information sur le domaine de longueur d'onde dans le header du fichier
lrefCN = float(loaded_files['corps_noir_4000K.fits'].header['CRVAL1']) # longueur d'onde de référence en nm
nbCN   = int(loaded_files['corps_noir_4000K.fits'].header['NAXIS1'])   # nombre de canaux d'échantillonnage
stepCN = float(loaded_files['corps_noir_4000K.fits'].header['CD1_1'])  # écart entre deux échantillon en nm

waveCN = np.linspace(lrefCN-nbCN*stepCN/2,lrefCN+nbCN*stepCN/2,nbCN) # Ici le pixel qui sert de référence est au milieu de l'image, 
# la longueur d'onde minimale est donc lambda_min= lambda_ref - nb_lambda*dlambda/2
# de même la longueur d'onde maximale est lambda_max= lambda_ref + nb_lambda*dlambda/2

CN4000K_func = interpolate.interp1d(waveCN, CN4000K)

CN4000K = CN4000K_func(wave)

In [ ]:
# CODE A COMPLETER : calculer le master_flat_5s


**Relever et enregistrer la valeur moyenne du flat moyen. Construire le master flat normalisé et l'enregistrer sous la forme "nom_prenom_exo2_MASTER_FLAT_NORM.dat".**

In [ ]:
# CODE A COMPLETER : sauvegarder et afficher la valeur moyenne du flat moyen


##### **Question 20 (1 pt) :**
Commenter l'allure du master flat normalisé.

In [ ]:
# CODE A COMPLETER


> <font color='blue'> Répondre ici : 

### **2.3 Mesure de la transmission atmosphérique - Comparaison aux données professionnelles**

Jusqu'à présent nous n'avons pas pris en compte l'impact de l'atmosphère pour réaliser le master flat. Dans cette partie nous allons réduire le spectre d'une étoile : Capella, afin de déterminer la correction à appliquer pour corriger de la transmission atmosphérique : $T_{atmo}$. 
<br>
Pour cela nous allons utiliser un spectre de référence accessible à partir d'un observatoire virtuel.

#### **2.3.1 Réduction des spectres individuels**

Dans un premier temps nous allons faire la réduction complète du spectre de l'étoile Capella qui se trouve dans le dossier "exercice2".

##### **Question 21 (3 pts) :**
Effectuer la réduction sur l'étoile Capella (5 spectres individuels *capella_5s_1.fits*, *capella_5s_2.fits*, etc...) en utilisant le master flat normalisé et le bon master dark. 
<br>
Diviser le spectre réduit par le temps d'exposition total (en seconde). 
<br>
Afficher dans une même fenêtre un spectre brut et le spectre réduit.
<br>
**Comparer le spectre avant et après réduction** (pensez à normaliser vos spectre pour pouvoir les comparer). 

In [ ]:
# CODE A COMPLETER : Faire la réduction complète des spectres de capella_5s


In [ ]:
# CODE A COMPLETER : Afficher le spectre réduit et un spectre brut


> <font color='blue'> Répondre ici : 

##### **Question 22 (1 pt) :**
Enregistrer le spectre réduit en lui donnant comme nom : 
<br>
*nom_prenom_exo2_SPECTRE_CAPELLA_REDUIT.dat*

In [ ]:
# CODE A COMPLETER : sauvegarder et afficher la valeur moyenne du flat moyen


#### **2.3.2 Détermination de la transmission atmosphérique**

Nous allons maintenant utiliser un spectre professionnel provenant d'une base de données pour étalonner la transmission atmosphérique.

##### **Question 23 (2.5 pts) :**
Afficher le spectre *capella_calib_angstrom*. La résolution du spectre est-elle plus ou moins grande que celle de vos spectres ?
<br>
**Attention au domaine de longueur d'onde du spectre de calibration qui est différent de celui de vos spectres, ainsi que les unités !** Nommez le nouveau tableau de longueur d'onde *wavecalib*.

In [ ]:
# CODE A COMPLETER : Afficher le spectre de calibration de capella


> <font color='blue'> Répondre ici : 

##### **Question 24 (1 pt) :**
Comment peut-on obtenir $T_{atmo}$ à partir de votre spectre réduit et du spectre de calibration ?
<br>
**Faire valider votre méthodologie par un encadrant.**
<br>
Calculer puis afficher la transmission atmosphérique $T_{atmo}$ en fonction de la longueur d'onde jusqu'à 750 nm.

In [ ]:
# On fait une interpolation de notre spectre de calibration pour que l'on calcul les points sur notre plage de longueur d'onde.
capella_calib_func = interpolate.interp1d(wavecalib,capella_calib)

# On détermine l'indice du tableau wave le plus proche qui correspond la longueur d'onde 750 nm.
# On se limite à 750 nm car le spectre de calibration ne va pas aussi loin en longueur d'onde que nos spectres
# et que les données du spectre de référence sont abérrantes au dessus de 750 nm.
ind_750 = np.argmin(np.abs(wave-750.))
new_capella_calib = capella_calib_func(wave[:ind_750])

In [ ]:
# CODE A COMPLETER : Calculer et afficher T_atmo


> <font color='blue'> Répondre ici : 

##### **Question 25 (1.5 pts) :**
Commenter la figure obtenue.

> <font color='blue'> Répondre ici : 

##### **Question 26 (1 pt) :**
Enregistrer cette estimation de la transmission atmosphérique sous le nom *nom_prenom_exo2_T_atmo.dat*.

In [ ]:
# CODE A COMPLETER : sauvegarder et afficher la valeur moyenne du flat moyen


#### **2.3.3 Exploitation du spectre**

Nous allons maintenant analyser le spectre de Capella réduit et en déduire certaines propriétes. Pour être plus précis, il faudrait corriger vos spectres de la transmission atmosphérique ce que nous n'allons pas faire ici.

**Température de Capella**

Le rayonnement des étoiles est décrit par ce que l'on appelle un corps noir en physique,
c'est-à-dire un corps qui émet infiniment moins qu'il n'absorbe d'énergie. Ainsi, la forme d'un
spectre est d'abord déterminée par la température de l'objet. La loi de Wien donne la variation
du rayonnement d'un corps noir en fonction de la longueur d'onde. Une formule dérivée de cette
loi permet de relier la température ($T$) d'un objet à la longueur d'onde où le maximum du
rayonnement est émis ($\lambda_{max}$) :

\begin{equation}
    \label{eq_2} \tag{2}
    T \times \lambda_{max} = 2,898 \cdot 10^{-3} \text{[SI]}
\end{equation} 

##### **Question 27 (1 pt) :**
Quel est l'unité de la constante $2,898 \cdot 10^{-3}$, aussi appelée constante de Wien ?

> <font color='blue'> Répondre ici : 

##### **Question 28 (2 pts) :**
Déterminer à quelle longueur d'onde est le maximum du spectre avec ses incertitudes. Estimer une température pour Capella ainsi que son type spectral en vous aidant de la figure 1 ci dessous.

**Les types spectraux des étoiles en fonction de leur température.**
![alt text](ressources/classe-spectrale-etoiles.jpg "Les types spectraux des étoiles en fonction de leur température.")

Phrase mnémotechnique pour se souvenir des types spectraux : " **O**h **B**e **A** **F**ine **G**irl/**G**uy,**K**iss **M**e" !

**Courbes de corps noir en fonction de la longueur d'onde pour plusieurs températures.**
![alt text](ressources/Blackbody_emission.png "Courbes de corps noir en fonction de la longueur d'onde pour plusieurs températures.")


> <font color='blue'> Répondre ici : 

**identification de raies**

On peut aussi identifier quelques raies typiques de ce type d'étoiles.
<br>Afficher sur une même figure votre spectre réduit ainsi que l'identification des raies contenue dans *PSL.ids*.

In [ ]:
# lecture des listes de raies et remplissage du dictionnaire

raies_list = [] # liste contenant les raies du fichier 'PSL.ids' sous la forme ['element','position']

with open('./Data_TP7_CPES2/Exercice2/PSL.ids','r') as f:
    for line in f.readlines()[10:]:
        # skip 10 first lines
        try:
            elements = line.replace('\t',' ').replace('*','').split(' ')
            element_name = elements[-1].replace('\n','')
            element_pos  = float(elements[0])
            raies_list.append([element_name,element_pos]) 
        except Exception as e:
            print(e)

In [ ]:
# CODE A COMPLETER


##### **Question 29 (2 pts) :**
Identifier les raies les plus importantes, de quelles espèces chimiques proviennent-elles ?

> <font color='blue'> Répondre ici : 

##### **Question 30 (1 pt) :**
A votre avis d'où proviennent-elles ?

> <font color='blue'> Répondre ici : 

## **Conclusion**

> <font color='blue'> Répondre ici : 